In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

# Default datastore
default_store = ws.get_default_datastore() 

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your CPU cluster
cluster_name = "SBCCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, hence using it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

# Hyperdrive classification job

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal, Run
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0,1),
        '--max_iter': choice(200, 250, 400, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=10, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#args = ['--C', 0.75,
#        '--max_iter', 200]

cwd = os.getcwd()
src = ScriptRunConfig(source_directory=cwd,
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env
                      )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [ ]:
%%capture
hyperdrive_run.wait_for_completion(show_output=False)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

In [ ]:
# Get your best run and save the model from that run.
import joblib

# from azureml.core import Run
from azureml.train.hyperdrive import HyperDriveRun

best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [ ]:
best_run.get_details()['runDefinition']['arguments']

In [ ]:
best_run.get_metrics()['Accuracy']

In [ ]:
best_run_filelist = best_run.get_file_names()

## AutoML Pipeline job

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

%tb
import os
import urllib
import train

from azureml.data.dataset_factory import TabularDatasetFactory

cwd = os.getcwd()
data_folder = os.path.join(cwd,'data')
os.makedirs(data_folder, exist_ok=True)

URL = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

urllib.request.urlretrieve(URL, filename=os.path.join(data_folder, 'bankmarketing_train.csv'))

ds = TabularDatasetFactory.from_delimited_files(URL)


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# Split data 70%-30% into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=1234)

training_df = pd.concat((x_train,y_train),1)
test_df = pd.concat((x_test,y_test),1)

# create csv files for train/test and upload to datastore 
training_df.to_csv(data_folder+"/train.csv")
test_df.to_csv(data_folder+"/test.csv")

default_store.upload_files([data_folder+"/train.csv"], 
                           target_path = 'udacitySBC', 
                           overwrite = True, 
                           show_progress = True)

default_store.upload_files([data_folder+"/test.csv"], 
                           target_path = 'udacitySBC', 
                           overwrite = True, 
                           show_progress = True)

#Create and register datasets

training_ds = Dataset.Tabular.from_delimited_files(default_store.path('udacitySBC/train.csv'))
test_ds = Dataset.Tabular.from_delimited_files(default_store.path('udacitySBC/test.csv'))

In [ ]:
from azureml.train.automl import AutoMLConfig, utilities

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

# Classification metrics (all possible)
class_metrics = utilities.get_primary_metrics('classification')

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target = aml_compute,
    primary_metric=class_metrics[1],
    featurization = 'auto',
    training_data=training_ds,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

# Define AutML step
from azureml.pipeline.steps import AutoMLStep

trainWithAutomlStep = AutoMLStep(name='AutoML_Classification',
                                 automl_config=automl_config,
                                 allow_reuse=True)

# Build and run the pipeline

from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline_steps = [trainWithAutomlStep]

pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
pipeline_run = exp.submit(pipeline, regenerate_outputs=False)

In [ ]:
RunDetails(pipeline_run).show()

### Explore the results 
#### View the details of the AutoML run

In [ ]:
from azureml.train.automl.run import AutoMLRun

# get_steps() returns the steps from latest to first

autoML_steps = pipeline_run.get_steps()

# Get the automl run as its the last step in the pipeline
for step in autoML_steps:
    automl_step_run_id = step.id
    print(step.name)
    print(automl_step_run_id)
    break

automl_run = AutoMLRun(experiment = exp, run_id=automl_step_run_id)

In [ ]:
# Retrieve and save your best automl model.

# Before proceeding, need to wait for the run to complete.
pipeline_run.wait_for_completion(show_output=False)

best_run, fitted_model = automl_run.get_output()

In [ ]:
best_run.get_metrics()

#### Test the best fitted model

In [ ]:
y_predict = fitted_model.predict(x_test)

# y_actual =  y_test.values.tolist()

# display(pd.DataFrame({'Actual':y_actual, 'Predicted':y_predict}).head(5))

### Register best model

In [ ]:
#model_path 

best_run.get_file_names()

model = best_run.register_model(model_name='udacityprojectSBC_AutoML', model_path='outputs')